In [0]:
%run ./01-config

In [0]:
class HistoryLoader():
    def __init__(self, env):
        Conf = Config()
        self.landing_zone = Conf.base_data_path + "/raw"
        self.catalog = env
        self.db_name = 'ecommerce_db_bz'

    def load_date_lookup(self):        
        print(f"Đang nạp bảng date_lookup_bz...", end='')    
        from pyspark.sql.types import IntegerType, ShortType, ByteType
        from pyspark.sql.functions import col, to_timestamp
        csv_path = self.landing_zone + "/date.csv"
        df = spark.read.option("header", "true").csv(csv_path)
        df_processed = df.select(
            col("date_key").cast(IntegerType()),
            to_timestamp(col("full_date"), "dd-MM-yy HH:mm"),
            col("day_of_week").cast(ByteType()),
            col("day_num_in_month").cast(ByteType()),
            col("day_num_overall").cast(ShortType()),
            col("day_name"),
            col("day_abbrev"),
            col("weekday_flag"),
            col("week_num_in_year").cast(ByteType()),
            col("week_num_overall").cast(ShortType()),
            to_timestamp(col("week_begin_date"), "dd-MM-yy HH:mm"),
            col("week_begin_date_key").cast(IntegerType()),
            col("month").cast(ByteType()),
            col("month_num_overall").cast(ShortType()),
            col("month_name"),
            col("month_abbrev"),
            col("quarter").cast(ByteType()),
            col("year").cast(ShortType()),
            col("yearmo").cast(IntegerType()),
            col("fiscal_month").cast(ByteType()),
            col("fiscal_quarter").cast(ByteType()),
            col("fiscal_year").cast(ShortType()),
            col("last_day_in_month_flag"),
            to_timestamp(col("same_day_year_ago_date"), "dd-MM-yy HH:mm")
        )
        table_name = f"{self.catalog}.{self.db_name}.date_lookup_bz"
        df_processed.write.mode("overwrite").insertInto(table_name)
        
    def load_history(self):
        import time
        start = int(time.time())
        print(f"\nTiến hành nạp dữ liệu cũ ...")
        self.load_date_lookup()
        print(f"Dữ liệu cũ được nạp thành công trong {int(time.time()) - start} giây")

In [0]:
HL = HistoryLoader('dev')
HL.load_history()